In [2]:
!pip install lingpy
!pip install Bio
!pip install numpy
!pip install pandas
!pip install random
!pip install multiprocessing
!pio install re
!pip install ast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 26.6 MB/s eta 0:00:00
  Created w

In [3]:
import lingpy
import pandas as pd
import re
from lingpy import *
from Bio import pairwise2
import numpy as np
import ast

def cleanASJP(word):
    """
    Adapted and shortened from Jaeger's cleaning in alignment.py.
    """
    word = re.sub(r"[,\%\"~]", "", word)
    word = re.sub(r"(.)(.)(.)\$", r"\2", word)
    return word.replace('~', '')
#
#
# data = pd.read_csv('albanoRomance.wordpairs.csv')
# data = data.drop(columns=['PMI'])
# data['word1'] = data['word1'].apply(cleanASJP)
# data['word2'] = data['word2'].apply(cleanASJP)
#
# print(data.head())
#
# pmi dict
pmi_df = pd.read_csv('pmi-albanoRomance.csv', index_col=0)
pmi_dict = {}
for i in range(pmi_df.shape[0]):
    for j in range(pmi_df.shape[1]):
        pmi_dict[(pmi_df.columns[j], pmi_df.index[i])] = pmi_df.iloc[i, j]
#
#
#
# def align_word_pairs(df):
#     alignments = []
#
#     for index, row in df.iterrows():
#         seqA = row['word1']
#         seqB = row['word2']
#         try:
#             nw_aln = align.pairwise.nw_align(seqA, seqB, scorer=pmi_dict)
#             sw_aln = align.pairwise.sw_align(seqA, seqB, scorer=pmi_dict)
#             pw_aln = align.pairwise.pw_align(seqA, seqB, mode="dialign")
#         except:
#             continue
#
#         alignments.append({
#             'concept1': row['concept1'],
#             'language1': row['language1'],
#             'word1': row['word1'],
#             'ID1': row['ID1'],
#             'concept2': row['concept2'],
#             'language2': row['language2'],
#             'word2': row['word2'],
#             'ID2': row['ID2'],
#             'nw_alignment': nw_aln,
#             'sw_alignment': sw_aln,
#             'pw_alignment': pw_aln
#         })
#
#     return pd.DataFrame(alignments)
#
#
# alignments_df = align_word_pairs(data)
# alignments_df.to_csv('aligned_lingpy.csv', index=False)


# turchin only returns 0 or 1, not useful


def convert_SW(alignment, keep_score=False):
    GAP_SYMBOL = '-'
    if isinstance(alignment, str):
        try:
            parsed_input = ast.literal_eval(alignment)
        except (ValueError, SyntaxError) as e:
            raise ValueError("Wrong alignment format.") from e
    elif isinstance(alignment, tuple):
        parsed_input = alignment
    else:
        raise ValueError("Unsupported alignment format.")

    lists1, lists2, score = parsed_input
    max_cols = max(len(lists1), len(lists2))
    output_lists1 = []
    output_lists2 = []

    for i in range(max_cols):
        seqA = lists1[i] if i < len(lists1) else []
        seqB = lists2[i] if i < len(lists2) else []
        lenA = len(seqA)
        lenB = len(seqB)
        max_len = max(lenA, lenB)
        padded_seqA = ''.join(seqA) + GAP_SYMBOL * (max_len - lenA)
        padded_seqB = ''.join(seqB) + GAP_SYMBOL * (max_len - lenB)
        output_lists1.append(padded_seqA)
        output_lists2.append(padded_seqB)
    final_seqA = ''.join(output_lists1)
    final_seqB = ''.join(output_lists2)

    if keep_score:
        return final_seqA, final_seqB, score

    return final_seqA, final_seqB

2024-08-16 15:29:43,861 [INFO] Compiling diacritics and vowels...
2024-08-16 15:29:43,865 [INFO] Diacritics and sound classes were successfully compiled.
2024-08-16 15:29:43,866 [INFO] Compiling model <asjp>...
2024-08-16 15:29:43,868 [INFO] ('ǃ', '!')
2024-08-16 15:29:43,868 [INFO] ('ǂ', '!')
2024-08-16 15:29:43,870 [INFO] ('ǁ', '!')
2024-08-16 15:29:43,870 [INFO] ('ǀ', '!')
2024-08-16 15:29:43,871 [INFO] ('ʘ', '!')
2024-08-16 15:29:43,872 [INFO] ('gǃ', '!')
2024-08-16 15:29:43,873 [INFO] ('gǂ', '!')
2024-08-16 15:29:43,874 [INFO] ('gǁ', '!')
2024-08-16 15:29:43,874 [INFO] ('gǀ', '!')
2024-08-16 15:29:43,875 [INFO] ('gʘ', '!')
2024-08-16 15:29:43,876 [INFO] ('ǃŋ', '!')
2024-08-16 15:29:43,878 [INFO] ('ǂŋ', '!')
2024-08-16 15:29:43,879 [INFO] ('ǁŋ', '!')
2024-08-16 15:29:43,880 [INFO] ('ǀŋ', '!')
2024-08-16 15:29:43,880 [INFO] ('ʘŋ', '!')
2024-08-16 15:29:43,883 [INFO] ('!', '!')
2024-08-16 15:29:43,884 [INFO] ('|', '!')
2024-08-16 15:29:43,885 [INFO] ('g!', '!')
2024-08-16 15:29:43,88

In [15]:
# Results are saved in the pairwise_alignment folder, along with the ipynb used to execute.
import random as pyrandom

pyrandom.seed(12345)

from numpy import *
import numpy as np
random.seed(12345)
import pandas as pd
import re
from Bio import pairwise2
from multiprocessing import Process,Manager
from lingpy import *
import lingpy as lingpy
import ast
ncores = 50




# Function: nexCharOutput
# Description:
## This function takes a character array, a list of rownames
## and the name of the output nexus file as input
## and writes the character matrix into a nexus file.
## Missing entries are assumed to be coded as "-1"

def nexCharOutput(chMtx, names, outfile, datatype='STANDARD'):
    f = open(outfile, 'w')
    f.write('#NEXUS\n\n')
    f.write('BEGIN DATA;\n')
    f.write('DIMENSIONS ntax=' + str(len(chMtx)) + ' NCHAR=' + str(len(chMtx.T)) + ';\n')
    f.write('FORMAT DATATYPE=' + datatype + ' GAP=? MISSING=- interleave=yes;\n')
    f.write('MATRIX\n\n')
    txLgth = max(map(len, names))
    for i in range(len(chMtx)):
        f.write(names[i].ljust(txLgth + 2))
        for ch in chMtx[i]:
            if ch == -1:
                ch = '-'
            else:
                ch = str(ch)
            f.write(ch)
        f.write('\n')
    f.write('\n;\n\nEND;\n')
    f.close()


data = pd.read_csv('dataset.tab',
                   index_col=0, na_filter=False, sep='\t')
data = data[data.wls_gen.isin(['ROMANCE', 'ALBANIAN'])]
data = data[data.index != 'LATIN']
concepts100 = array(data.columns[9:])
nEntries = data[concepts100].apply(lambda x: sum(x != '')).sort_values()
concepts = nEntries.index[-40:]
data = data[concepts]
taxa = array(data.index)

dataWL = pd.DataFrame([(c,l,cleanASJP(w))
                       for c in concepts
                       for l in taxa
                       for w in data[c][l].split(',')
                       if data[c][l]!=''],
                      columns = ['concept','language','word'])

dataWL = dataWL.drop_duplicates(['concept','language'])

training = pd.DataFrame()
for c in concepts:
    cData = dataWL[dataWL.concept==c]
    cTraining = pd.DataFrame([cData.loc[[i,j]].word.values
                              for i in cData.index
                              for j in cData.index
                              if i<j])
    training = pd.concat([training, cTraining], ignore_index=True)

sounds = unique(concatenate(list(map(list,dataWL.word.values))))


def recalculate_pmi(alignment, gp1, gp2):
    """Recalculate the PMI score based on gap penalties."""
    print(alignment)
    score = alignment[2]
    gap_count = 0
    seqA, seqB = alignment[0], alignment[1]
    for i in range(len(seqA)):
        if seqA[i] == '-' or seqB[i] == '-':
            gap_count += 1
            if gap_count == 1:
                score -= 1
                score += gp1
            else:
                score -= 1
                score += gp2
    return score


def sw_align_lingpy(w, scorer=False, gp1=None, gp2=None, keep_scores=False):
    x, y = w
    sw_aln = lingpy.align.pairwise.sw_align(x, y, scorer=scorer)
    sw_aln = convert_SW(sw_aln, keep_score=keep_scores)
    if gp1 and gp2:
        sw_aln = (sw_aln[0], sw_aln[1], recalculate_pmi(sw_aln, gp1, gp2))
    return sw_aln



manager = Manager()
return_dict = manager.dict()
packages = array_split(training.values,ncores)


def doWork(i,pck):
    results = []
    for p in pck:
        alignment = sw_align_lingpy(p)
        if alignment:
            results.append(alignment)
    print(f'Package {i}: Number of successful alignments - {len(results)}')
    if results:
        return_dict[i] = vstack(results)
        print(f'Package {i}: Results added to return_dict')
    else:
        print(f"Warning: No successful alignments for package {i}")


jobs = []
for i,pck in enumerate(packages):
    p = Process(target=doWork,args=(i,pck))
    p.start()
    jobs.append(p)
for p in jobs:
    p.join()
if return_dict:
  alg0 = vstack(return_dict.values())
else:
  print("Error: the dict is empty")

# count alignment frequencies
sFreqs = pd.crosstab(alg0[:,0],alg0[:,1])
sFreqs = sFreqs.reindex(sounds,fill_value=0).T
sFreqs = sFreqs.reindex(sounds,fill_value=0).T
# symmetrize
sFreqs = sFreqs.copy()+sFreqs.copy().T
# add-1 smoothing
sFreqs += 1
sProbs = sFreqs/sFreqs.sum().sum()

# extract relative sound frequencies
soundOccurrences = concatenate([list(w) for w in dataWL.word.values])
soundProbabilities = pd.value_counts(soundOccurrences,normalize=True)[sounds]
pmi0 = (np.log(sProbs).copy()-np.log(soundProbabilities.to_numpy().reshape(-1, 1))).T-np.log(soundProbabilities.to_numpy().reshape(-1, 1))


pmi0Dict = {(s1, s2): pmi0[s1][s2]
            for s1 in sounds for s2 in sounds}


def align_no_gaps(w, pmi_dict, gp1, gp2, th=-Inf, mode=None):
    """w: pair of ASJP strings
    pmiDict: dictionary of logodds (=PMI scores)
    gp1,gp2: gap penalties (non-positive)
    th: threshold; all pairs with a PMI-score <th will be ignored
    returns: array of pairwise alignment, with gaps removed"""
    a = sw_align_lingpy(w, scorer=pmi_dict, keep_scores=True)
    if len(a) == 0: return zeros((0, 2))
    algn = []
    # check if the alignment score is below the threshold, handling potential NaN
    if a[2] < th if not isnan(a[2]) else False:
        return zeros((0, 2))
    # extract aligned pairs without gaps
    seqA, seqB = a[0], a[1]  # unpacking the aligned sequences
    l = len(seqA)
    aaa = [[seqA[i], seqB[i]] for i in range(l)]
    algn += [x for x in aaa if not '-' in x]
    return array(algn)



def alignStar(crp, pmi_dict, gp1, gp2, th, align_func):
    packages = array_split(crp, ncores)
    manager = Manager()
    return_dict = manager.dict()

    def doWork(i, pck):
        return_dict[i] = vstack([align_no_gaps(w, pmi_dict, gp1, gp2, th, mode=align_func)
                                 for w in pck])

    jobs = []
    for i, pck in enumerate(packages):
        p = Process(target=doWork, args=(i, pck))
        p.start()
        jobs.append(p)
    for p in jobs:
        p.join()
    return vstack(return_dict.values())


def swalignStar(crp, pmi_dict, gp1, gp2, th):
    return alignStar(crp, pmi_dict, gp1, gp2, th, 'sw')


gp1 = -2.49302792222
gp2 = -1.70573165621
th = 4.4451

pmiDict = pmi0Dict.copy()
for i in range(10):
    sw_alg = swalignStar(training.values, pmiDict, gp1, gp2, th)
    sFreqs = pd.crosstab(sw_alg[:, 0], sw_alg[:, 1])
    sFreqs = sFreqs.reindex(sounds, fill_value=0).T
    sFreqs = sFreqs.reindex(sounds, fill_value=0).T
    sFreqs = sFreqs.copy() + sFreqs.copy().T
    sFreqs += 1
    sProbs = sFreqs / sFreqs.sum().sum()
    pmi = (np.log(sProbs).copy()-np.log(soundProbabilities.to_numpy().reshape(-1, 1))).T-np.log(soundProbabilities.to_numpy().reshape(-1, 1))
    #pmi = (np.log(sProbs).copy() - np.log(soundProbabilities)).T - np.log(soundProbabilities)
    pmiDict = {(s1, s2): pmi[s1][s2]
                for s1 in sounds for s2 in sounds}
    print(f"PMI dict: {pmiDict}")
    pmi.to_csv(f'pmi-albanoRomance-sw.csv')

dataWL.to_csv('albanoRomanceASJP-sw.csv', index=False)

sc = pd.DataFrame(index=taxa)
for c in concepts:
    cData = dataWL[dataWL.concept == c]
    cTaxa = cData.language.unique()
    cWords = pd.Series([''.join(cData[cData.language == l].word.values)
                        for l in cTaxa],
                       index=cTaxa)
    cMtx = pd.DataFrame([[int(s in cWords[l]) for s in sounds]
                         for l in cTaxa],
                        index=cTaxa)
    cMtx = cMtx.reindex(taxa, fill_value='-')
    sc = pd.concat([sc, cMtx], axis=1)

nexCharOutput(sc.values, sc.index, 'albanoRomanceSC-sw.nex', 'restriction')


Package 0: Number of successful alignments - 1040
Package 0: Results added to return_dictPackage 3: Number of successful alignments - 1040
Package 1: Number of successful alignments - 1040

Package 2: Number of successful alignments - 1040Package 3: Results added to return_dictPackage 4: Number of successful alignments - 1040Package 1: Results added to return_dict



Package 5: Number of successful alignments - 1040Package 4: Results added to return_dictPackage 2: Results added to return_dictPackage 6: Number of successful alignments - 1040



Package 5: Results added to return_dictPackage 6: Results added to return_dict

Package 7: Number of successful alignments - 1040
Package 7: Results added to return_dict
Package 8: Number of successful alignments - 1040
Package 8: Results added to return_dict
Package 9: Number of successful alignments - 1040
Package 10: Number of successful alignments - 1040
Package 9: Results added to return_dictPackage 10: Results added to return_dict

Package 

<ipython-input-15-ada54524dced>:151: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  soundProbabilities = pd.value_counts(soundOccurrences,normalize=True)[sounds]


PMI dict: {(' ', ' '): 5.993126968780281, (' ', '*'): 4.99167842856582, (' ', '3'): 3.6377913104007074, (' ', '5'): 5.4657720430630805, (' ', '8'): 6.080138345769912, (' ', 'C'): 5.9216680047981365, (' ', 'E'): 2.9173519875527547, (' ', 'L'): 3.967174112051432, (' ', 'N'): 4.4816694647063855, (' ', 'S'): 4.539693304822762, (' ', 'T'): 4.4890495720040064, (' ', 'Z'): 4.417590608021862, (' ', 'a'): 1.1701608378045751, (' ', 'b'): 2.8150731384323358, (' ', 'c'): 5.993126968780281, (' ', 'd'): 2.3000895045176812, (' ', 'e'): 1.286604288531902, (' ', 'f'): 2.836126547630168, (' ', 'g'): 3.970843840940394, (' ', 'h'): 4.95167309395212, (' ', 'i'): 1.568280336923471, (' ', 'j'): 4.894514680112172, (' ', 'k'): 2.997394695226291, (' ', 'l'): 2.9525806794364984, (' ', 'm'): 2.14653176867459, (' ', 'n'): 1.3171877119039825, (' ', 'o'): 1.2210309114762086, (' ', 'p'): 4.013768885657429, (' ', 'q'): 7.7848864380083365, (' ', 'r'): 1.556375434417153, (' ', 's'): 1.8238810983850628, (' ', 't'): 1.831